In [1]:
import pandas as pd
import numpy
import csv
import matplotlib.pyplot as plt

In [2]:
df_pop = pd.read_csv('../data/dataset/API_SP.POP.TOTL_DS2_en_csv_v2.csv')

In [3]:
df_2016 = pd.read_csv('../data/dataset/HIGH_AGLINK_2016_18102017105443042.csv')

In [4]:
df_pop.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101220.0,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,...,26616792.0,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0
2,Angola,AGO,"Population, total",SP.POP.TOTL,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,...,20997687.0,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,82683.0,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0


In [5]:
df_2016.head()

,COUNTRY,Country,COMMODITY,Commodity,VARIABLE,Variable,TIME,Time,Value,Flag Codes,Flags
0,AUS,Australia,WT,Wheat,QP,Production,2015,2015,23981.80800,NaN,NaN
1,AUS,Australia,WT,Wheat,QP,Production,2016,2016,24908.13334,NaN,NaN
2,AUS,Australia,WT,Wheat,QP,Production,2017,2017,25596.13971,NaN,NaN
3,AUS,Australia,WT,Wheat,QP,Production,2018,2018,26135.14009,NaN,NaN
4,AUS,Australia,WT,Wheat,QP,Production,2019,2019,26317.25042,NaN,NaN


In [6]:
df_2016['Country'].value_counts()

WORLD                          3619
United States                  3542
European Union-28              3289
Canada                         3234
Japan                          3201
China                          3201
Argentina                      3190
DEVELOPED COUNTRIES            3190
OECD countries                 3157
Brazil                         3157
Israel                         3146
Mexico                         3135
Australia                      3135
Sudan                          3124
Korea                          3080
Haiti                          3069
Russian Federation             3058
New Zealand                    3058
DEVELOPING COUNTRIES           3014
India                          3014
Iran                           3003
Non-OECD                       3003
Indonesia                      2992
Thailand                       2992
Chile                          2992
Malaysia                       2981
Turkey                         2981
Colombia                    

In [7]:
df_2016.head(5)

,COUNTRY,Country,COMMODITY,Commodity,VARIABLE,Variable,TIME,Time,Value,Flag Codes,Flags
0,AUS,Australia,WT,Wheat,QP,Production,2015,2015,23981.80800,NaN,NaN
1,AUS,Australia,WT,Wheat,QP,Production,2016,2016,24908.13334,NaN,NaN
2,AUS,Australia,WT,Wheat,QP,Production,2017,2017,25596.13971,NaN,NaN
3,AUS,Australia,WT,Wheat,QP,Production,2018,2018,26135.14009,NaN,NaN
4,AUS,Australia,WT,Wheat,QP,Production,2019,2019,26317.25042,NaN,NaN


# Function for ratio creation (averaging value for chosen year and commodity) 

In [8]:
def prepare_ratio(df, year, commodity, df_pop):
    df_filt = df[df['Time'] == year]
    df_filt_2 = df_filt[df_filt['Commodity'] == commodity][df_filt['Variable'] == 'Human consumption per capita']
    df_consump = df_filt_2.drop(['Flags', 'Flag Codes', 'VARIABLE', 'Time', 'COMMODITY', 'Commodity', 'Variable'], axis=1)
    df_pop_filt = df_pop[['Country Code', str(year)]]
    df_pop_filt = df_pop_filt.rename(columns={'Country Code': 'COUNTRY'})
    result = df_pop_filt.merge(df_consump, left_on='COUNTRY', right_on='COUNTRY', how='inner')
    result = result.sort_values('Value')
    result['ratio'] = result['Value'] / result[str(year)]
    result.sort_values('ratio')
    return result

In [9]:
def plot_values(df, year, lst, df_pop):
    df_plot = pd.DataFrame({'COUNTRY' : df['COUNTRY'].unique()})
    for i in lst:
        df1 = prepare_ratio(df, year, i, df_pop)
        df1[i] = df1['ratio']
        df1 = df1.drop(['2015', 'Country', 'TIME', 'Value', 'ratio'], axis=1)        
        df_plot = df_plot.merge(df1, left_on='COUNTRY', right_on='COUNTRY', how='inner')
    df_plot = df_plot.set_index(df_plot['COUNTRY'])
    del df_plot['COUNTRY']
    return df_plot